In [1]:
import json
import pandas as pd

In [46]:
df = pd.read_json (r'channelData.json')

col_names = ['title',
             'channelId',
             'description',
             'viewCount',
             'subscriberCount',
             'videoCount']
total_df= pd.DataFrame()
for index, row in df.iterrows():
    items=df['items'][index][0]
    snippet = items['snippet']

    title=snippet['title']
    channelId=items['id']
    publishedAt=snippet['publishedAt']
    description=snippet['description']
    viewCount=items['statistics']['viewCount']
    subscriberCount=items['statistics']['subscriberCount']
    videoCount=items['statistics']['videoCount']

    data = {'title':title, 
    'channelId': channelId, 
    'publishedAt': publishedAt,
    'description': description,
    'viewCount': viewCount,
    'subscriberCount':subscriberCount,
    'videoCount':videoCount
    }
    df_generated = pd.DataFrame(data, index=[index])
    total_df= total_df.append(df_generated)
total_df


,title,channelId,publishedAt,description,viewCount,subscriberCount,videoCount
0,Alex Clark,UCsDmESjqNPukDmVnuneLrqw,2009-12-02T20:40:49Z,Funny vids about my life. I'm thankful for th...,471863645,4280000,317
1,Alex Meyers,UCY6Ij8zOds0WJEeqCLOnqOQ,2015-07-09T08:02:42Z,I make cartoons and sometimes video essays abo...,472343511,2870000,256
2,WowRightMeow,UClN24S5-fc7LD0JJW0FdGWw,2016-08-13T22:14:53Z,"Yo! I'm Jamie, a dorky human from Wales. Did a...",29689338,537000,33
3,Andrei Terbea,UC3iNdSV_RQU7DHTGpqEW96w,2013-04-30T19:30:17Z,Doing animated commentary and solving crimes w...,292059762,2860000,136
4,Billy But Better,UC_52i6mlrzOZs_NEuSepzAg,2014-04-08T02:13:16Z,"Yes, it's a foot.\n\nSocial Medias:\nTwitter -...",12165987,260000,23
5,Casually Explained,UCr3cBLTYmIK9kY0F_OdFWFQ,2015-12-07T22:39:35Z,If you're looking for the answers to life's qu...,374495141,3580000,81
6,CircleToonsHD,UCV6g95OBbVtFmN9uiJzkFqQ,2012-12-15T03:09:11Z,"My name is Christian, but many know me as Circ...",523362138,2240000,249
7,CurtRichy,UCgCKYs56-LKEPGQ99DzqQOg,2014-11-26T18:55:15Z,"Yo, welcome to the channel. Lower your standar...",18027473,341000,97
8,CypherDen,UCOwxx9VnEnlFKt5EB70KTzQ,2012-06-07T03:28:32Z,"Hi, I'm Den.\n\nI breathe air. It's nice.",140659288,1490000,63
9,Daidus,UC2Ds30pkifFVD0CE08wF50g,2015-11-01T21:11:06Z,My animation videos are fast-paced so you migh...,83815630,1550000,33
